In [1]:
!pip install jiwer


**Word Error Rate (WER) is defined as:**
   $$
   WER = \frac{S + D + I}{N}
   $$
   where:
   - $S =$ number of substitutions (word is different in reference text from predicted)
   - $D =$ number of deletions (word in the reference text is missing from the predicted)
   - $I =$ number of insertions (extra word is present in the predicted that is not in the reference text)
   - $N =$ total number of words in the reference sentence

   Reference: [WER](https://thepythoncode.com/article/calculate-word-error-rate-in-python)

# **Preprocessing**

In [2]:
def normalize_arabic(text):
    text = text.replace("أ", "ا")
    text = text.replace("إ", "ا")
    text = text.replace("آ", "ا")
    text = text.replace("ى", "ي")
    text = text.replace("ؤ", "و")
    text = text.replace("ئ", "ي")
    return text

In [3]:
import re
def remove_diacritics(text):
    arabic_diacritics = re.compile("""
         ّ |
         َ |
         ً |
         ُ |
         ٌ |
         ِ |
         ٍ |
         ْ |
         ـ
     """, re.VERBOSE)
    return re.sub(arabic_diacritics, '', text)

In [4]:
import string

def preprocess(filename):
    with open(filename, "r", encoding="utf-8") as f:
        # Read all lines, strip spaces, ignore empty lines
        lines = [line.strip() for line in f if line.strip()]
        # Join lines into a single string
        text = " ".join(lines)
    text = normalize_arabic(text)
    text = remove_diacritics(text)
    # Remove English and Arabic punctuation (which is after the +)
    all_punct = string.punctuation + "؟؛،"
    text = text.translate(str.maketrans('', '', all_punct))
    # Lowercase English letters
    text = ''.join([c.lower() if c.isascii() else c for c in text])
    # Remove extra spaces
    text = " ".join(text.split())
    return text

# **Get Word Error Rate**

## English WER without noise

In [5]:
reference = preprocess("english/original.txt")
predicted_otter = preprocess("english/otter-english.txt")
predicted_google_docs = preprocess("english/google-english.txt")
predicted_google_pro = preprocess("english/google-pro-english.txt")
predicted_whisper = preprocess("english/whisper-english.txt")

In [6]:
from jiwer import wer
WER = wer(reference, predicted_google_pro)
print(f"Word Error Rate Google Pro: {WER:.2f}")
WER = wer(reference, predicted_google_docs)
print(f"Word Error Rate Google Docs: {WER:.2f}")
WER = wer(reference, predicted_otter)
print(f"Word Error Rate Otter: {WER:.2f}")
WER = wer(reference, predicted_whisper)
print(f"Word Error Rate Whisper: {WER:.2f}")

Word Error Rate Google Pro: 0.18
Word Error Rate Google Docs: 0.20
Word Error Rate Otter: 0.20
Word Error Rate Whisper: 0.20


## English WER with noise

In [7]:
predicted_otter = preprocess("noise_english/otter-noise-english.txt")
predicted_google_docs = preprocess("noise_english/google-noise-english.txt")
predicted_google_pro = preprocess("noise_english/google-pro-noise_english.txt")
predicted_whisper = preprocess("noise_english/whisper-english-noise.txt")

In [8]:
WER = wer(reference, predicted_google_pro)
print(f"Word Error Rate Google Pro: {WER:.2f}")
WER = wer(reference, predicted_google_docs)
print(f"Word Error Rate Google Docs: {WER:.2f}")
WER = wer(reference, predicted_otter)
print(f"Word Error Rate Otter: {WER:.2f}")
WER = wer(reference, predicted_whisper)
print(f"Word Error Rate Whisper: {WER:.2f}")

Word Error Rate Google Pro: 0.21
Word Error Rate Google Docs: 0.19
Word Error Rate Otter: 0.17
Word Error Rate Whisper: 0.29


## Arabic WER without Noise

In [9]:
reference = preprocess("arabic/original.txt")
predicted_google_docs = preprocess("arabic/google-arabic.txt")
predicted_google_pro = preprocess("arabic/google-pro-arabic.txt")
predicted_whisper = preprocess("arabic/whisper-arabic.txt")

In [10]:
WER = wer(reference, predicted_google_pro)
print(f"Word Error Rate Google Pro: {WER:.2f}")
WER = wer(reference, predicted_google_docs)
print(f"Word Error Rate Google Docs: {WER:.2f}")
WER = wer(reference, predicted_whisper)
print(f"Word Error Rate Whisper: {WER:.2f}")

Word Error Rate Google Pro: 0.21
Word Error Rate Google Docs: 0.21
Word Error Rate Whisper: 0.19


## Arabic WER with Noise

In [11]:
predicted_google_docs = preprocess("noise_arabic/google-arabic.txt")
predicted_google_pro = preprocess("noise_arabic/google-pro-noise_arabic.txt")
predicted_whisper = preprocess("noise_arabic/whisper-arabic-noise.txt")

In [12]:
WER = wer(reference, predicted_google_pro)
print(f"Word Error Rate Google Pro: {WER:.2f}")
WER = wer(reference, predicted_google_docs)
print(f"Word Error Rate Google Docs: {WER:.2f}")
WER = wer(reference, predicted_whisper)
print(f"Word Error Rate Whisper: {WER:.2f}")

Word Error Rate Google Pro: 0.22
Word Error Rate Google Docs: 0.22
Word Error Rate Whisper: 0.21


## Code-switching without Noise

In [13]:
reference = preprocess("mixed/original.txt")
predicted_google_pro = preprocess("mixed/google-pro-mixed.txt")
predicted_whisper = preprocess("mixed/whisper-mixed.txt")

In [14]:
WER = wer(reference, predicted_google_pro)
print(f"Word Error Rate Google Pro: {WER:.2f}")
WER = wer(reference, predicted_whisper)
print(f"Word Error Rate Whisper: {WER:.2f}")

Word Error Rate Google Pro: 0.55
Word Error Rate Whisper: 0.77


## Code-switching with Noise

In [15]:
predicted_google_pro = preprocess("noise_mixed/google-pro-noise_mixed.txt")
predicted_whisper = preprocess("noise_mixed/whisper-mixed-noise.txt")

In [16]:
WER = wer(reference, predicted_google_pro)
print(f"Word Error Rate Google Pro: {WER:.2f}")
WER = wer(reference, predicted_whisper)
print(f"Word Error Rate Whisper: {WER:.2f}")

Word Error Rate Google Pro: 1.24
Word Error Rate Whisper: 0.28
